In [2]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel
import pickle
import os

In [3]:
pandarallel.initialize(progress_bar=False, nb_workers=8)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## Data preparation

In [6]:
data = pd.read_csv("../../data/data_cleaned_sentences_phases_2020-04-16.csv", sep='|', converters={'sentences': pd.eval})

In [7]:
data["title"] = data["title"].parallel_apply(lambda title: title if isinstance(title, str) else "") 
data["text"] = data.parallel_apply(lambda row: " ".join([sent for sent in [row["title"]] + row["sentences"]]), axis=1)

In [ ]:
data = data.reset_index().rename(columns={"index": "id"})
data = data[["id", "text", "Label"]]

In [8]:
row = data.loc[19]
print(row.title)
print(row.sentences)
print("--------------------")
print(row.text)
print(row.Label)

haber
['filistinde yahudi yerleşimcilerin polis korumasında mescidi aksanın avlusuna girmesi gerginliğe neden oldu', 'yahudi yerleşimcilerden oluşan kişilik grup israil polisinin koruması altında mescidi aksanın avlusuna girdi', 'bunun üzerine birgrup filistinli gönüllü kadın yahudi yerleşimcilere tepki göstererek oturma eylemi düzenledi israilli kolluk güçleri olayı protesto eden filistinlilere gerçek mermilerle saldırdı', 'filistinlilerden gerçek ise plastik mermilerle yaralanırken onlanca kişi ise atılan göz yaşartıcı gazdan etkilendi', 'israil polisinin dün sabah saatlerinden itibaren aksanın kapılarında güvenlik önlemlerini arttırdığı ifade edildi', 'öte yandan dün israil gazzede hamasm silahlı kanadı izzeddin elkassam tugayiarının eğitim alanına hava saldırısı düzenledi', 'saldırıda ölen ya da yaralanan olmadı', 'imi lifi mm buj']
--------------------
haber filistinde yahudi yerleşimcilerin polis korumasında mescidi aksanın avlusuna girmesi gerginliğe neden oldu yahudi yerleşimci

In [ ]:
pickle.dump(data, open("../data/data_id-text-label_2022-10-14.pkl", "wb"))

## Huggingface custom dataset

In [2]:
data = pickle.load(open("../data/data_id-text-label_2022-10-14.pkl", "rb"))
data

,id,text,Label
0,0,sahte polislerin kuryesi yakalandı sahte polis...,hate
1,1,kürt üz ama hain değiliz kürtüz ama hain değil...,hate
2,2,suriyeli gelinden altın vurgunu kuyumcuda altı...,hate
3,3,mustafa nevruz sınacı mustafa nevruz sınacı lg...,hate
4,4,mustafa nevruz sınacı mustafa nevruz sınacı ya...,hate
...,...,...,...
25061,25061,amnesty ınternational ve global ahlaksızlık do...,hate
25062,25062,çanakkale asla unutulmamalı llnutturulmamalı ç...,hate
25063,25063,sömürü projesi olarak bop btp genel başkanı pr...,hate
25064,25064,doğruluş zeminimiz helali bir millet istiklali...,hate


In [13]:
data

,id,date,pub_name,type,title,content,Label,sentences,text,phase
0,0,05 Ekim 2015 Pazartesi,akşam,ulusal,sahte polislerin kuryesi yakalandı,sahte polislerin kuryesi yakalandı antalya'da ...,hate,[sahte polislerin kuryesi yakalandı antalyada ...,sahte polislerin kuryesi yakalandı sahte polis...,train
1,1,11 Eylül 2015 Cuma,akşam,ulusal,kürt üz ama hain değiliz,kürt'üz ama hain değiliz suriye sınırında devr...,hate,[kürtüz ama hain değiliz suriye sınırında devr...,kürt üz ama hain değiliz kürtüz ama hain değil...,test
2,2,25 Eylül 2015 Cuma,akşam,ulusal,suriyeli gelinden altın vurgunu,kuyumcuda altın alırken fotoğraf çektirdi. sur...,hate,"[kuyumcuda altın alırken fotoğraf çektirdi, su...",suriyeli gelinden altın vurgunu kuyumcuda altı...,test
3,3,07 Eylül 2015 Pazartesi,anayurt,ulusal,mustafa nevruz sınacı,"mustafa nevruz sınacı lgercek. abd'li yahudi, ...",hate,"[mustafa nevruz sınacı lgercek, abdli yahudi b...",mustafa nevruz sınacı mustafa nevruz sınacı lg...,train
4,4,21 Eylül 2015 Pazartesi,anayurt,ulusal,mustafa nevruz sınacı,"mustafa nevruz sınacı yazıyor, gercek. abd'li ...",hate,"[mustafa nevruz sınacı yazıyor gercek, abdli y...",mustafa nevruz sınacı mustafa nevruz sınacı ya...,train
...,...,...,...,...,...,...,...,...,...,...
25010,25061,02 Mayıs 2014 Cuma,yeni asya,hepsi,amnesty ınternational ve global ahlaksızlık,doğu veya batı s. bulut@saidnursi. de amnesty ...,hate,"[doğu veya batı, de amnesty ınternational ve g...",amnesty ınternational ve global ahlaksızlık do...,train
25011,25062,26 Mart 2014 Çarşamba,yeni konya,hepsi,çanakkale asla unutulmamalı llnutturulmamalı,"çanakkale, asla unutulmamalı, llnutturulmamalı...",hate,[çanakkale asla unutulmamalı llnutturulmamalı ...,çanakkale asla unutulmamalı llnutturulmamalı ç...,val
25012,25063,04 Nisan 2014 Cuma,yeni mesaj,hepsi,sömürü projesi olarak bop,btp genel başkanı prof. dr. haydar bas ın kale...,hate,"[btp genel başkanı prof, haydar bas ın kalemin...",sömürü projesi olarak bop btp genel başkanı pr...,train
25013,25064,24 Şubat 2014 Pazartesi,yeni mesaj,hepsi,doğruluş zeminimiz helali bir millet istiklali...,prof. dr. nurullah çetin doğruluş zeminimiz: '...,hate,[nurullah çetin doğruluş zeminimiz helali bir ...,doğruluş zeminimiz helali bir millet istiklali...,train


In [11]:
import torch
from collections import Counter
from sklearn.model_selection import train_test_split
from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_metric

In [35]:
class HDVDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, idxs):
        self.label_encodings = {"not_hate": 0, "hate": 1}
        self.encodings = encodings
        self.labels = [self.label_encodings[label] for label in labels]
        self.idxs = idxs

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [30]:
train_idxs, train_texts, train_labels = list(data.loc[data["phase"] == "train", "id"].values), list(data.loc[data["phase"] == "train", "text"].values), list(data.loc[data["phase"] == "train", "Label"].values)
val_idxs, val_texts, val_labels = list(data.loc[data["phase"] == "val", "id"].values), list(data.loc[data["phase"] == "val", "text"].values), list(data.loc[data["phase"] == "val", "Label"].values)
test_idxs, test_texts, test_labels = list(data.loc[data["phase"] == "test", "id"].values), list(data.loc[data["phase"] == "test", "text"].values), list(data.loc[data["phase"] == "test", "Label"].values)

In [31]:
print(f"Training dist: {Counter(train_labels)}")
print(f"Validation dist: {Counter(val_labels)}")
print(f"Test dist: {Counter(test_labels)}")

Training dist: Counter({'hate': 10074, 'not_hate': 9944})
Validation dist: Counter({'hate': 1257, 'not_hate': 1243})
Test dist: Counter({'hate': 1255, 'not_hate': 1242})


In [32]:
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-128k-uncased")

In [33]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [36]:
train_dataset = HDVDataset(train_encodings, train_labels, train_idxs)
val_dataset = HDVDataset(val_encodings, val_labels, val_idxs)
test_dataset = HDVDataset(test_encodings, test_labels, test_idxs)

## Evaluation metrics

In [37]:
prec = load_metric("precision")
rec = load_metric("recall")
acc = load_metric("accuracy")
f1 = load_metric("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    result = {}
    for mtrc in [prec, rec, acc, f1]:
        mtrc_result = mtrc.compute(predictions=predictions, references=labels)
        result.update(mtrc_result)
    return result

## Huggingface models

In [12]:
%env WANDB_PROJECT=hdv_hate_speech
%env WANDB_LOG_MODEL=true
%env WANDB_WATCH=all
%env WANDB_NOTEBOOK_NAME=berturk

env: WANDB_PROJECT=hdv_hate_speech
env: WANDB_LOG_MODEL=true
env: WANDB_WATCH=all
env: WANDB_NOTEBOOK_NAME=berturk


In [42]:
#import wandb
from transformers import Trainer, TrainingArguments

In [14]:
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find berturk.
wandb: Currently logged in as: nlpboun (use `wandb login --relogin` to force relogin)


True

In [38]:
results_path = "../experiments/results/"
logs_path = "../experiments/logs/"

In [39]:
model = AutoModelForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-128k-uncased", num_labels=2)

Downloading: 100%|██████████| 706M/706M [03:47<00:00, 3.26MB/s] 
Some weights of the model checkpoint at dbmdz/bert-base-turkish-128k-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceCl

In [43]:
training_args = TrainingArguments(
    output_dir=os.path.join(results_path, "berturk_128K_arute"),               # output directory
    num_train_epochs=2,                                                  # total number of training epochs
    per_device_train_batch_size=4,                                       # batch size per device during training
    per_device_eval_batch_size=4,                                        # batch size for evaluation
    warmup_steps=500,                                                    # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                                                   # strength of weight decay
    logging_dir=os.path.join(results_path, "berturk_128K_arute"),              # directory for storing logs
    logging_steps=20,
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    save_steps=1000,
    learning_rate=1e-05,
    run_name="berturk_128K_uncased_lre-5",
    evaluation_strategy="steps",
    eval_steps=500,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,                                                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                                                  # training arguments, defined above
    train_dataset=train_dataset,                                         # training dataset
    eval_dataset=val_dataset,                                            # evaluation dataset
    compute_metrics=compute_metrics
)

In [44]:
trainer.train()

/home/arute/miniconda3/envs/nlp_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 20018
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 10010


Step,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
500,0.575400,0.470825,0.778609,0.828162,0.795200,0.802621
1000,0.326600,0.417024,0.866092,0.879873,0.871200,0.872928
1500,0.241000,0.452083,0.893755,0.876691,0.885600,0.885141


***** Running Evaluation *****
  Num examples = 2500
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 4
Saving model checkpoint to ../experiments/results/berturk_128K_arute/checkpoint-1000
Configuration saved in ../experiments/results/berturk_128K_arute/checkpoint-1000/config.json
Model weights saved in ../experiments/results/berturk_128K_arute/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2500
  Batch size = 4


KeyboardInterrupt: 

In [18]:
trainer.train()

/home/tabilab/anaconda3/envs/hate_env/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20052
  Num Epochs = 2
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 10026
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Precision,Recall,Accuracy,F1
500,0.507400,0.470382,0.742204,0.847310,0.774631,0.791282
1000,0.412100,0.487355,0.869938,0.883703,0.874751,0.876766
1500,0.331500,0.453500,0.880620,0.898734,0.887515,0.889585
2000,0.263200,0.537385,0.930530,0.805380,0.871560,0.863444
2500,0.674700,0.489855,0.929856,0.818038,0.877144,0.870370
3000,0.593000,0.378214,0.902478,0.893196,0.897487,0.897813
3500,0.425200,0.367395,0.888298,0.924842,0.903470,0.906202
4000,0.372100,0.365355,0.894453,0.918513,0.904268,0.906323
4500,0.552500,0.460086,0.850035,0.959652,0.894296,0.901524
5000,0.224900,0.378452,0.916599,0.895570,0.906262,0.905962


***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4
Saving model checkpoint to ../experiments/results/berturk_128K/checkpoint-1000
Configuration saved in ../experiments/results/berturk_128K/checkpoint-1000/config.json
Model weights saved in ../experiments/results/berturk_128K/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4
Saving model checkpoint to ../experiments/results/berturk_128K/checkpoint-2000
Configuration saved in ../experiments/results/berturk_128K/checkpoint-2000/config.json
Model weights saved in ../experiments/results/berturk_128K/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4
***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4
Saving model checkpoint to ../experiments/results/berturk_128K

TrainOutput(global_step=10026, training_loss=0.38614428408388934, metrics={'train_runtime': 1770.0672, 'train_samples_per_second': 22.657, 'train_steps_per_second': 5.664, 'total_flos': 1.055180576415744e+16, 'train_loss': 0.38614428408388934, 'epoch': 2.0})

## Evaluation (validation)

In [19]:
eval_results = trainer.evaluate(val_dataset)
eval_results

***** Running Evaluation *****
  Num examples = 2507
  Batch size = 4


{'eval_loss': 0.3649916350841522,
 'eval_precision': 0.9016018306636155,
 'eval_recall': 0.935126582278481,
 'eval_accuracy': 0.9158356601515756,
 'eval_f1': 0.9180582524271845,
 'eval_runtime': 22.3554,
 'eval_samples_per_second': 112.143,
 'eval_steps_per_second': 28.047,
 'epoch': 2.0}

## Testing

In [45]:
preds_dict = trainer.predict(test_dataset)
predictions = preds_dict.predictions
predictions = np.argmax(predictions, axis=1)
print(f"Preds: {predictions}\n GT's: {preds_dict.label_ids}")
print(preds_dict.metrics)

***** Running Prediction *****
  Num examples = 2497
  Batch size = 4


Preds: [0 1 1 ... 1 1 1]
 GT's: [1 1 1 ... 1 1 1]
{'test_loss': 0.4906734824180603, 'test_precision': 0.8367931281317108, 'test_recall': 0.9314741035856574, 'test_accuracy': 0.8742490989187024, 'test_f1': 0.8815987933634993, 'test_runtime': 17.5851, 'test_samples_per_second': 141.996, 'test_steps_per_second': 35.542}


In [50]:
class HDVDatasetTest(torch.utils.data.Dataset):
    def __init__(self, texts_idxs, labels, tokenizer):
        self.label_encodings = {"not_hate": 0, "hate": 1}
        self.rev_label_encodings = {0: "not_hate", 1: "hate"}
        
        self.texts, self.idxs = list(np.array(texts_idxs)[:, 0]), list(np.array(texts_idxs)[:, 1])
        self.encodings = tokenizer(self.texts, truncation=True, padding=True)
        self.labels = [self.label_encodings[label] for label in labels]
        self.preds = []

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
    def _get_preds_with_idx(self):
        df_preds = pd.DataFrame(data={"idx": self.idxs, "prediction": self.preds})
        df_preds["prediction"] = df_preds["prediction"].map(self.rev_label_encodings)
        return df_preds

In [50]:
texts_idxs, labels = list(data[["text", "id"]].values), list(data["Label"].values)
train_texts_idxs_2, val_texts_idxs_2, train_labels_2, val_labels_2 = train_test_split(texts_idxs, labels, stratify=labels, test_size=.2, shuffle=True, random_state=17)
val_texts_idxs_2, test_texts_idxs_2, val_labels_2, test_labels_2 = train_test_split(val_texts_idxs_2, val_labels_2, stratify=val_labels_2, test_size=.5, shuffle=True, random_state=17)

In [51]:
test_dataset_idx = HDVDatasetTest(test_texts_idxs_2, test_labels_2, tokenizer)

In [27]:
preds_dict = trainer.predict(test_dataset)
predictions = preds_dict.predictions
predictions = np.argmax(predictions, axis=1)
print(f"Preds: {predictions}\n GT's: {preds_dict.label_ids}")
print(preds_dict.metrics)

***** Running Prediction *****
  Num examples = 2507
  Batch size = 4


Preds: [1 0 1 ... 1 0 1]
 GT's: [1 0 1 ... 1 0 1]
{'test_loss': 0.42105528712272644, 'test_precision': 0.8891419893697798, 'test_recall': 0.9271575613618369, 'test_accuracy': 0.9050658157159952, 'test_f1': 0.9077519379844963, 'test_runtime': 23.0777, 'test_samples_per_second': 108.633, 'test_steps_per_second': 27.169}


In [28]:
predictions

array([1, 0, 1, ..., 1, 0, 1])

In [52]:
preds_dict_2 = trainer.predict(test_dataset_idx)
predictions_2 = preds_dict_2.predictions
predictions_2 = np.argmax(predictions_2, axis=1)
print(f"Preds: {predictions_2}\n GT's: {preds_dict_2.label_ids}")
print(preds_dict_2.metrics)

***** Running Prediction *****
  Num examples = 2507
  Batch size = 4


Preds: [1 0 1 ... 1 0 1]
 GT's: [1 0 1 ... 1 0 1]
{'test_loss': 0.42105528712272644, 'test_precision': 0.8891419893697798, 'test_recall': 0.9271575613618369, 'test_accuracy': 0.9050658157159952, 'test_f1': 0.9077519379844963, 'test_runtime': 23.2055, 'test_samples_per_second': 108.035, 'test_steps_per_second': 27.019}


In [53]:
test_dataset_idx.preds = predictions_2
df_preds = test_dataset_idx._get_preds_with_idx()

In [54]:
df_preds

,idx,prediction
0,3973,hate
1,16849,not_hate
2,9186,hate
3,3072,hate
4,10346,hate
...,...,...
2502,3672,hate
2503,18446,not_hate
2504,3657,hate
2505,19535,not_hate


## Report to csv

In [58]:
df_label_preds = pd.merge(data, df_preds, left_on="id", right_on="idx", how="right").drop("idx", axis=1)
df_label_preds.to_excel("../outputs/labels_preds_berturk_2022-04-14.xlsx", index=False)

In [59]:
df_label_preds

,id,text,Label,prediction
0,3973,haber seyfullah koyuncu freddy mercurynin aske...,hate,hate
1,16849,kilisede hz fatıma nın doğumu kutlandı kilised...,not_hate,not_hate
2,9186,yunanlıların verdiği zararları anlatan resmi d...,hate,hate
3,3072,itı hr it serdar çalışkan yaptığı açıklamada ş...,hate,hate
4,10346,mültecileri dövüp geri gönderdiler mültecileri...,hate,hate
...,...,...,...,...
2502,3672,saitiyor su rl yıu ur bpfjmkmmiami rj fiil il ...,hate,hate
2503,18446,bu işbirllfii türkiye ye örnek oucak bu işbirl...,not_hate,not_hate
2504,3657,mersinde işlenen cinayetle ilgili suriyeli tut...,hate,hate
2505,19535,siparişle kurulan proje örgütlerdir siparişle ...,not_hate,not_hate


In [60]:
data_raw = pd.read_csv("../data/data_cleaned_sentences_2020-04-10.csv", sep='|', converters={'sentences': pd.eval})

In [63]:
data_raw = data_raw.reset_index().rename(columns={"index": "id"})

In [65]:
df_data_and_preds = pd.merge(data_raw, df_preds, left_on="id", right_on="idx", how="right").drop("idx", axis=1)

In [68]:
df_data_and_preds.to_excel("../outputs/data_and_preds_2022-04-14.xlsx", index=False)

In [67]:
sum(df_data_and_preds["Label"] == df_data_and_preds["prediction"]) / df_data_and_preds.shape[0]

0.9050658157159952

In [70]:
Counter(val_labels)

Counter({'hate': 1264, 'not_hate': 1243})

In [63]:
df = pd.read_excel("../outputs/data_and_preds_2022-04-14.xlsx")

In [64]:
(sum(df["Label"] == df["prediction"]) - len(remove_id_from_test)) / (df.shape[0] - len(remove_id_from_test))

0.9046856227472968

## Duplicate rows

In [10]:
id_excel = pd.read_excel("../outputs/data_and_preds_2022-04-14.xlsx")["id"].values.tolist()

In [11]:
list(np.array(test_texts_idxs_2)[:, 1]) == id_excel

True

In [12]:
set.intersection(set(np.array(test_texts_idxs_2)[:, 1]), set(np.array(train_texts_idxs_2)[:, 1]))

set()

In [48]:
first_pair = data[data[["text"]].duplicated(keep="first")].sort_values("text").id.values.tolist()
last_pair = data[data[["text"]].duplicated(keep="last")].sort_values("text").id.values.tolist()

In [49]:
pairs = [[first_pair[i], last_pair[i]] for i in range(len(first_pair))]
pairs

[]

In [40]:
train_idxs, val_idxs, test_idxs = list(np.array(train_texts_idxs_2)[:, 1]), list(np.array(val_texts_idxs_2)[:, 1]), list(np.array(test_texts_idxs_2)[:, 1])

In [50]:
def detect_duplicate_rows(train_idxs, test_idxs, pairs):
    remove_id_from_test = []
    for pair in pairs:
        if pair[0] in train_idxs and pair[1] in train_idxs:
            print(pair, " is all in train set!")
        elif pair[0] in train_idxs and pair[1] in test_idxs:
            print(f"{pair[0]} in train, {pair[1]} in test, please delete from test!!!")
            remove_id_from_test.append(pair[1])
        elif pair[1] in train_idxs and pair[0] in test_idxs:
            print(f"{pair[1]} in train, {pair[0]} in test, please delete from test!!!")
            remove_id_from_test.append(pair[0])
        elif pair[0] in test_idxs and pair[1] in test_idxs:
            print(f"{pair[0]} in test, {pair[1]} in test, please delete one of them from test!!!")
            remove_id_from_test.append(pair[1])
    remove_test_indices = []
    for remove_id in remove_id_from_test:
        remove_test_indices.append(test_idxs.index(remove_id))
    remove_test_indices
    return remove_id_from_test, remove_test_indices

In [51]:
remove_id_from_test = detect_duplicate_rows(train_idxs, test_idxs, pairs)
remove_id_from_test

([], [])

In [52]:
remove_id_from_val = detect_duplicate_rows(train_idxs, val_idxs, pairs)
remove_id_from_val

([], [])

## Inference

In [66]:
model = AutoModelForSequenceClassification.from_pretrained("../experiments/results/berturk_128K/checkpoint-10000")

In [67]:
class HDVDatasetTest(torch.utils.data.Dataset):
    def __init__(self, texts_idxs, labels, tokenizer, remove_idxs):
        self.label_encodings = {"not_hate": 0, "hate": 1}
        self.rev_label_encodings = {0: "not_hate", 1: "hate"}
        
        self.texts, self.idxs = list(np.array(texts_idxs)[:, 0]), list(np.array(texts_idxs)[:, 1])
        print(len(self.idxs))
        self.texts = [text for i, text in enumerate(self.texts) if i not in remove_idxs]
        self.idxs = [idx for i, idx in enumerate(self.idxs) if i not in remove_idxs]
        self.encodings = tokenizer(self.texts, truncation=True, padding=True)
        self.labels = [self.label_encodings[label] for i, label in enumerate(labels) if i not in remove_idxs]
        self.preds = []
        print(len(self.idxs))
        
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
    def _get_preds_with_idx(self):
        df_preds = pd.DataFrame(data={"idx": self.idxs, "prediction": self.preds})
        df_preds["prediction"] = df_preds["prediction"].map(self.rev_label_encodings)
        return df_preds

In [69]:
test_dataset_idx_cleaned = HDVDatasetTest(test_texts_idxs_2, test_labels_2, tokenizer, remove_test_indices)

2507
2497


In [75]:
trainer = Trainer(
    model=model,                                                         # the instantiated 🤗 Transformers model to be trained
    compute_metrics=compute_metrics
)

In [77]:
preds_dict_3 = trainer.predict(test_dataset_idx_cleaned)
predictions_3 = preds_dict_3.predictions
predictions_3 = np.argmax(predictions_3, axis=1)
print(f"Preds: {predictions_3}\n GT's: {preds_dict_3.label_ids}")
print(preds_dict_3.metrics)

***** Running Prediction *****
  Num examples = 2497
  Batch size = 8


Preds: [1 0 1 ... 1 0 1]
 GT's: [1 0 1 ... 1 0 1]
{'test_loss': 0.4224632978439331, 'test_precision': 0.8884644766997708, 'test_recall': 0.9266932270916335, 'test_accuracy': 0.9046856227472968, 'test_f1': 0.9071762870514819, 'test_runtime': 17.3161, 'test_samples_per_second': 144.201, 'test_steps_per_second': 18.076}
